# Azure OpenAI Service - Q&A with semantic answering tutorial

In this tutorial, you'll build a simple Q&A system, that can give semantic answers to questions. Three sample documents from the Azure documentation are provided. Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

In [25]:
from azure.identity import DefaultAzureCredential
import os
import json
import tiktoken
import openai
import numpy as np
from dotenv import load_dotenv, find_dotenv
from openai.embeddings_utils import cosine_similarity

# Load environment variables
load_dotenv(find_dotenv("credentials.env"))

# Configure OpenAI API
default_credential = DefaultAzureCredential()
token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
openai.api_type = "azure_ad"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('AOAI_ENDPOINT')
openai.api_key = token.token

# Define completion model, embedding model and encoding
COMPLETION_MODEL = 'text-davinci-003'
EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_ENCODING = 'cl100k_base'
EMBEDDING_CHUNK_SIZE = 8000

# initialize tiktoken for encoding text
encoding = tiktoken.get_encoding(EMBEDDING_ENCODING)

Next, let's read the documents in `samples/*.json`, which are our sample documents. The `content` section is the interesting piece of information for this tutorial:

```json
{
  "content": "\n# What is Azure OpenAI?\n\nThe ...",
  "product_name": "cognitive-services",
  "title": "Azure Cognitive Services",
  "description": "Apply advanced language models to variety of use cases with the Azure OpenAI service",
  "topic": "overview",
  "date": "11/07/2022"
}
```

In [5]:
# list all files in the "data" directory
samples_dir = os.path.join(os.getcwd(), "../data/docs")
sample_files = os.listdir(samples_dir)

# read content field from each file and append it to documents, and remove and newlines (better for embeddings later)
documents = []
for file in sample_files:
    with open(os.path.join(samples_dir, file), "r") as f:
        content = json.load(f)["content"]
        content = content.replace("\n", " ")
        content = content.replace("  ", " ")
        documents.append(content)

# print some stats about the documents
print(f"Loaded {len(documents)} documents")
for doc in documents:
    num_tokens = len(encoding.encode(doc))
    print(f"Content: {doc[:80]}... \n---> Tokens: {num_tokens}\n")

Loaded 3 documents
Content:  # What is Azure OpenAI? The Azure OpenAI service provides REST API access to Op... 
---> Tokens: 1891

Content:  # What is Azure Cognitive Services Translator? Translator Service is a cloud-ba... 
---> Tokens: 739

Content:  # What is conversational language understanding? Conversational language unders... 
---> Tokens: 1341



Now that we have all documents loaded, we can embed them using our embedding model:

In [6]:
# Create embeddings for all docs
embeddings = [openai.Embedding.create(input=doc, engine=EMBEDDING_MODEL)["data"][0]["embedding"] for doc in documents]

# print some stats about the embeddings
for e in embeddings:
    print(len(e))

1536
1536
1536


Now that we have our embeddings, we can try to ask some questions and see if it retrieves the correct document. You can try the following questions:

* what is azure openai service?
* can translator be fine tuned?
* what is the difference between luis and clu?
* what is form recognizer? (should yield no result)

In [13]:
# create embedding for question
question = "How can I get access to GPT-3?"
qe = openai.Embedding.create(input=question, engine=EMBEDDING_MODEL)["data"][0]["embedding"]

# calculate cosine similarity between question and each document
similaries = [cosine_similarity(qe, e) for e in embeddings]

# Get the matching document, in this case we just use argmax of similarities
max_i = np.argmax(similaries)

# print some stats about the similarities
for i, s in enumerate(similaries):
    print(f"Similarity to {sample_files[i]} is {s}")
print(f"Matching document is {sample_files[max_i]}")

Similarity to overview_openai.json is 0.808711784272037
Similarity to overview_translator.json is 0.7090683100589205
Similarity to overview_clu.json is 0.7010604306231274
Matching document is overview_openai.json


In [15]:
# Generate a prompt that we use for completion, in this case we put the matched document and the question in the prompt
prompt = f""""
Content:
{documents[max_i]}
Please answer the question below using only the content from above. If you don't know the answer, say "I couldn't find the answer".
Question: {question}
Answer:"""

# get response from completion model
response = openai.Completion.create(
    engine=COMPLETION_MODEL,
    prompt=prompt,
    temperature=0.7,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
answer = response['choices'][0]['text']

# print the question and answer
print(f"Question was: {question}\nRetrieved answer was: {answer}")

InvalidRequestError: 2 is greater than the maximum of 1 - 'top_p'

Great, that worked. Now we should have a simple understanding how Q&A can work using OpenAI embeddings and completions. Next step would be:

* Chunking of longer documents (you might run into token limits for embeddings and the answering prompt)
* Usage of a vector database (pinecone, redis, etc.) to scale the search part to a larger amount of documents
* Evaluation of the top k results, instead of just the best matching document
* ...and a few more!